## Audiobook Business Case | Preprocessing

### 0. Extract the data from the csv

In [ ]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('3.0Audiobooks_data.csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:, 1:-1]  # all rows, all columns except the first and last (first for id, last for the target)
targets_all = raw_csv_data[:, -1]  # all rows, only the last column (the target)

### 1. Balance the dataset (# target=1 ~= # target=0)

In [ ]:
num_one_targets = int(np.sum(targets_all))  # count how many targets are 1
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### 2. Standardize the inputs

In [ ]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### 3. Shuffle the data

In [ ]:
shuffled_indices = np.arange(scaled_inputs.shape[0])  # create an array of indices
np.random.shuffle(shuffled_indices)  # shuffle the indices

shuffled_inputs = scaled_inputs[shuffled_indices]  # shuffle the inputs
shuffled_targets = targets_equal_priors[shuffled_indices]  # shuffle the targets

### 4. Split the dataset into train, validation, and test

In [ ]:
samples_count = shuffled_inputs.shape[0]  # number of samples

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets)/train_samples_count)  # print the number of targets=1, total samples, and the ratio of targets=1 in the training set
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets)/validation_samples_count)  # same for validation set
print(np.sum(test_targets), test_samples_count, np.sum(test_targets)/test_samples_count)  # same for test set

### 5. Save the three datasets in *.npz

In [ ]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)  # save the training set
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)  # save the validation set
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)  # save the test set